In [1]:
# Instala a biblioteca Groq, usada para acessar modelos de linguagem hospedados na Groq
!pip install -q groq

# Instala a biblioteca SentenceTransformers, usada para criar embeddings (representações vetoriais de textos)
!pip install -q sentence-transformers

# Instala a biblioteca Wikipedia-API, que permite buscar e manipular conteúdo da Wikipedia
!pip install -q wikipedia-api

# Instala/atualiza um conjunto de bibliotecas da LangChain e utilitários extras:
# - langchain-nvidia-ai-endpoints: integração da LangChain com modelos da NVIDIA
# - langchain-community: integrações mantidas pela comunidade (APIs, ferramentas, modelos)
# - langchain: framework principal para criar aplicações com LLMs
# - langgraph: extensão para estruturar fluxos conversacionais e agentes em forma de grafo
# - tavily-python: integração com a API de busca Tavily (busca na web)
# - beautifulsoup4: biblioteca para raspagem e análise de HTML
# - lxml: parser rápido para XML/HTML, usado junto com o BeautifulSoup
!pip install -qU langchain-nvidia-ai-endpoints langchain-community langchain langgraph tavily-python beautifulsoup4 lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 15.9 MB/s eta 0:00:00
ERROR: pip's

In [2]:
# Bibliotecas padrão
import json          # Para manipulação de dados em formato JSON
import os            # Para interagir com variáveis de ambiente e sistema de arquivos
import numpy as np   # Para cálculos numéricos e manipulação de arrays
import operator      # Para operadores úteis em comparações e ordenações

# Colab
from google.colab import userdata  # Para acessar secrets/variáveis de ambiente salvas no Colab

# Groq
from groq import Groq  # Cliente para acessar modelos de linguagem hospedados na Groq

# IPython
from IPython.display import Image, display  # Exibir imagens diretamente no notebook

# LangChain - utilitários principais
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Quebra textos em chunks para embeddings

# LangChain Community - loaders e ferramentas adicionais
from langchain_community.document_loaders import WebBaseLoader   # Carrega documentos a partir de páginas web
from langchain_community.tools.tavily_search import TavilySearchResults  # Ferramenta para buscar resultados na web via Tavily

# LangChain Core
from langchain_core.vectorstores import InMemoryVectorStore  # Vetoriza e armazena embeddings em memória
from langchain_core.messages import HumanMessage, SystemMessage  # Estrutura mensagens para interações com LLMs

# LangChain NVIDIA
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings, ChatNVIDIA
# - NVIDIAEmbeddings: gera embeddings usando modelos NVIDIA
# - ChatNVIDIA: interage com modelos de chat NVIDIA

# LangChain - estrutura de documentos
from langchain.schema import Document  # Representa documentos que serão usados no pipeline

# LangGraph
from langgraph.graph import END, StateGraph
# - END: marcador para finalização de um grafo
# - StateGraph: estrutura de grafos de estados para agentes/conversas

# Pydantic
from pydantic import BaseModel, Field  # Para definição de modelos de dados com validação

# Sentence Transformers
from sentence_transformers import SentenceTransformer  # Modelo de embeddings baseado em transformadores

# Tipagem avançada
from typing_extensions import TypedDict, List, Annotated, Literal
# Tipos auxiliares para melhorar clareza de estados e estruturas

# Wikipedia API
from wikipediaapi import Wikipedia  # API para buscar e manipular páginas da Wikipedia

In [6]:
# Ativa o tracing (rastreamento) da execução no LangSmith para monitorar os fluxos do LangChain
os.environ["LANGCHAIN_TRACING_V2"] = "True"

# Define o endpoint da API do LangSmith (plataforma de monitoramento do LangChain)
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

# Configura a chave de API do LangSmith (armazenada nos secrets do Colab)
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

# Nome do projeto dentro do LangSmith para organizar os rastreamentos e execuções
os.environ["LANGSMITH_PROJECT"] = "pr-crushing-nexus-98"

# Configura a chave de API da Tavily (serviço de busca na web), também obtida dos secrets do Colab
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

# Configura a chave de API da NVIDIA (para usar modelos de embeddings e chat NVIDIA)
os.environ["NVIDIA_API_KEY"] = userdata.get('NVIDIA_API_KEY')